In [2]:
# Workspace problem with several narrow gaps

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import os
import csv
from random import randint, random
import time

# (restrict tensorflow memory growth)
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

In [4]:
# neural network parameters
mb_size = 256
h_Q_dim = 512
h_P_dim = 512

c = 0
# learning rate
lr = 1e-4

# problem dimensions
dim = 7
dataElements = dim*3 + 16 + 16 # sample (7D), init (7D), goal (7D), cond (16 + 16 (table + box) points) //total = 53

z_dim = 3 # latent
X_dim = dim # samples
y_dim = dim # reconstruction of the original point
c_dim = dataElements - dim # dimension of conditioning variable

In [6]:
# define networks
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, X_dim])
c = tf.placeholder(tf.float32, shape=[None, 46])
    
# Q
inputs_Q = tf.concat(axis=1, values=[X,c])

dense_Q1 = tf.layers.dense(inputs=inputs_Q, units=h_Q_dim, activation=tf.nn.relu)
dropout_Q1 = tf.layers.dropout(inputs=dense_Q1, rate=0.5)
dense_Q2 = tf.layers.dense(inputs=dropout_Q1, units=h_Q_dim, activation=tf.nn.relu)

z_mu = tf.layers.dense(inputs=dense_Q2, units=z_dim) # output here is z_mu
z_logvar = tf.layers.dense(inputs=dense_Q2, units=z_dim) # output here is z_logvar

# P
eps = tf.random_normal(shape=tf.shape(z_mu))
z = z_mu + tf.exp(z_logvar / 2) * eps
inputs_P = tf.concat(axis=1, values=[z,c])

dense_P1 = tf.layers.dense(inputs=inputs_P, units=h_P_dim, activation=tf.nn.relu)
dropout_P1 = tf.layers.dropout(inputs=dense_P1, rate=0.5)
dense_P2 = tf.layers.dense(inputs=dropout_P1, units=h_P_dim, activation=tf.nn.relu)

y = tf.layers.dense(inputs=dense_P2, units=X_dim) # fix to also output y

# training
########### comment in the one with 0 weight and uncomment the other ###########
w = [[1, 1, 1, 1, 1, 1, 1]];
# w = [[1, 1, 1, 0, 0, 0]];
recon_loss = tf.losses.mean_squared_error(labels=X, predictions=y, weights=w)

# TODO: fix loss function for angles going around
kl_loss = 10**-4 * 2 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)

cvae_loss = tf.reduce_mean(kl_loss + recon_loss)

train_step = tf.train.AdamOptimizer(lr).minimize(cvae_loss)

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
it = 0;

In [8]:
saver = tf.train.Saver()
path_ = os.getcwd() + "/checkpoints_table_box/model1.ckpt"
print("path = ",path_)
# print("numTrain = ",numTrain)
try:
    saver.restore(sess, path_)
    print("Model Restored!!")
except Exception as e:
    print("Could not restore checkpoint!")
    print(e)
print(c,z)

('path = ', '/home/vernwalrahul/projects/LearningRoadmaps/cvae_learner/checkpoints_table_box/model1.ckpt')
INFO:tensorflow:Restoring parameters from /home/vernwalrahul/projects/LearningRoadmaps/cvae_learner/checkpoints_table_box/model1.ckpt
Model Restored!!
(<tf.Tensor 'Placeholder_1:0' shape=(?, 46) dtype=float32>, <tf.Tensor 'add:0' shape=(?, 3) dtype=float32>)


In [10]:
# create test_conditions from data dir
print(c,z)
def create_samples(G, directory):
    start = np.loadtxt(directory+"/start_node.txt")
    goal = np.loadtxt(directory+"/goal_node.txt")
    cond = np.loadtxt(directory+"/conditions.txt")
    def state_to_numpy(state):
        strlist = state.split()
        val_list = [float(s) for s in strlist]
        return np.array(val_list)
    
    # cond = cond.split(",")
    path_nodes = []
    i = 0
    c_samples = []
    with open(directory + "/path_nodes.txt", 'r') as file:
        lines  = file.readlines()
        for line in lines:
            line = line.strip('\n')
#             print(line)
#             print("\n\n")
            if(not line == '-1'):
                s = state_to_numpy(G.node[str(int(start[i]))]['state'])
                g = state_to_numpy(G.node[str(int(goal[i]))]['state'])
                path_nodes = str(line).split(",")
                # print(path_nodes)
                for path_node in path_nodes:
                    node_conf = state_to_numpy(G.node[path_node]['state'])
                    curr_node = np.array([])
                    # print("Data = ",node_conf, s, g, cond)
#                     print("\n")
                    curr_node = np.concatenate((s, g, cond))
                    c_samples.append(curr_node)
    return np.array(c_samples)
print("c, z = ",c, z)

(<tf.Tensor 'Placeholder_1:0' shape=(?, 46) dtype=float32>, <tf.Tensor 'add:0' shape=(?, 3) dtype=float32>)
('c, z = ', <tf.Tensor 'Placeholder_1:0' shape=(?, 46) dtype=float32>, <tf.Tensor 'add:0' shape=(?, 3) dtype=float32>)


In [11]:
num_viz = 300
c_final = []
print("c, z = ", c, z)
import networkx as nx
from random import choice
G = nx.read_graphml("graphs/modified_graph_dense.graphml")

for i in range(20):
    env_no = str(i)

    directory = "data/T1/" + env_no
    c_samples = np.float32(create_samples(G, directory))

    for i in range(num_viz):
        c_final.append(choice(c_samples))
    z_final = np.random.randn(num_viz,z_dim)

    c_final = np.array(c_final)
    print("c_final_dim = ",c_final.shape)
    print("z_dim = ",z_final.shape)
    # directly sample from the latent space (preferred, what we will use in the end)
    y_viz, z_viz = sess.run([y, z], feed_dict={z: z_final, c: c_final})

    print(y_viz)
    np.savetxt("output_data/output_samples_"+env_no+".txt", y_viz, delimiter=" ", fmt="%s")
    np.savetxt("output_data/conditions_"+env_no+".txt", np.loadtxt(directory+"/conditions.txt"))

('c, z = ', <tf.Tensor 'Placeholder_1:0' shape=(?, 46) dtype=float32>, <tf.Tensor 'add:0' shape=(?, 3) dtype=float32>)
('c_final_dim = ', (300, 46))
('z_dim = ', (300, 3))
[[ 3.4728906  -0.48682195  1.8256898  ...  1.446287   -0.10137868
  -0.02686952]
 [ 2.516064    0.83798426 -1.0701284  ... -0.2980822   1.0007374
  -0.18996678]
 [ 5.1585383   0.5598561   2.2731428  ...  0.09126262  0.46424645
  -1.0604553 ]
 ...
 [ 1.3808554   1.0298545   2.6271293  ... -1.1182524   0.72944415
   0.24952783]
 [ 1.3189683   0.10047657 -0.59389186 ...  0.08661107  1.4229033
  -1.6586992 ]
 [ 0.9692725  -0.47552127  2.2939506  ... -1.0771638   0.14892668
  -2.0914767 ]]


AttributeError: 'numpy.ndarray' object has no attribute 'append'